In [28]:
import pandas as pd
from pandas import DataFrame, Series
from datetime import date, datetime
from openpyxl import load_workbook

In [29]:
filename = "분개장_샘플.xlsx"

wb = load_workbook(filename)
ws = wb.active

In [30]:
colnames = [x.value for x in ws[4]]
print(colnames)

['일자', '전표번호', '계정코드', '계정과목', '적요', '차변', '대변', '구분', '거래처명']


In [31]:
collist = []
for row in ws.iter_rows(min_row=5, max_row=100000, max_col=len(colnames), values_only=True):
    if row[0] == "합 계":
        break
    row = list(row)
    if row[0] is not None:
        일자 = row[0].split('/')
        일자 = date(2024, int(일자[0]), int(일자[1]))
        전표번호 = row[1]
    row[0] = 일자
    row[1] = 전표번호
    if pd.isna(row[5]):
        row[5] = 0
    if pd.isna(row[6]):
        row[6] = 0
    collist.append(row)
journal = DataFrame(collist, columns=colnames)


In [32]:
journal

,일자,전표번호,계정코드,계정과목,적요,차변,대변,구분,거래처명
0,2024-01-02,00001,00001,자본금,자본금 입금,0,2000000000,차변,None
1,2024-01-02,00001,10300,보통예금,자본금 입금,2000000000,0,대변,하나은행
2,2024-01-03,00001,10300,보통예금,객실 매출,33000000,0,차변,하나은행
3,2024-01-03,00001,40100,객실수입,객실 매출,0,33000000,대변,호텔스닷컴
4,2024-01-03,00002,80100,고객용품비,고객용품비 지출,20000000,0,차변,ABC용품사
...,...,...,...,...,...,...,...,...,...
543,2024-05-29,00004,10300,보통예금,고객용품비 지출,0,16100000,대변,하나은행
544,2024-05-29,00005,80200,기타영업비용,영업비용E,15000000,0,차변,C사
545,2024-05-29,00005,10300,보통예금,영업비용E,0,15000000,대변,하나은행
546,2024-05-30,00001,10300,보통예금,객실 매출,29000000,0,차변,하나은행


In [33]:
category_table = pd.DataFrame([
        ['자본금', '자본', '자본금', '자본금'],
        ['보통예금', '유동자산', '현금및현금성자산', '보통예금'],
        ['객실수입', '매출', '매출', '객실수입'],
        ['고객용품비', '판매비와관리비', '고객용품비', '고객용품비'],
        ['기타영업비용', '판매비와관리비', '판매비용', '기타영업비용'],
        ['판매수수료', '판매비와관리비', '판매비용', '판매수수료'],
        ['세금과공과', '판매비와관리비', '기타판관비', '세금과공과'],
        ['수도광열비', '판매비와관리비', '기타판관비', '수도광열비'],
        ['급여', '판매비와관리비', '인건비', '급여'],
        ['노무용역비', '판매비와관리비', '인건비', '노무용역비'],
        ['이자수입', '영업외수익', '영업외수익', '이자수입']
    ],
    columns=['구분', '대분류', '중분류', '소분류']
)
category_table.set_index('구분', inplace=True)


In [34]:
journal['대분류'] = None
journal['중분류'] = None
journal['소분류'] = None

for idx in journal.index:
    key = journal.loc[idx, '계정과목']
    journal.loc[idx, '대분류'] = category_table.loc[key, '대분류']
    journal.loc[idx, '중분류'] = category_table.loc[key, '중분류']
    journal.loc[idx, '소분류'] = category_table.loc[key, '소분류']

In [35]:
journal = journal[
    [
        '일자', '전표번호', '대분류', '중분류', '소분류', '계정코드', 
        '계정과목', '적요', '차변', '대변', '구분', '거래처명'
        ]
    ]

In [36]:
journal

,일자,전표번호,대분류,중분류,소분류,계정코드,계정과목,적요,차변,대변,구분,거래처명
0,2024-01-02,00001,자본,자본금,자본금,00001,자본금,자본금 입금,0,2000000000,차변,None
1,2024-01-02,00001,유동자산,현금및현금성자산,보통예금,10300,보통예금,자본금 입금,2000000000,0,대변,하나은행
2,2024-01-03,00001,유동자산,현금및현금성자산,보통예금,10300,보통예금,객실 매출,33000000,0,차변,하나은행
3,2024-01-03,00001,매출,매출,객실수입,40100,객실수입,객실 매출,0,33000000,대변,호텔스닷컴
4,2024-01-03,00002,판매비와관리비,고객용품비,고객용품비,80100,고객용품비,고객용품비 지출,20000000,0,차변,ABC용품사
...,...,...,...,...,...,...,...,...,...,...,...,...
543,2024-05-29,00004,유동자산,현금및현금성자산,보통예금,10300,보통예금,고객용품비 지출,0,16100000,대변,하나은행
544,2024-05-29,00005,판매비와관리비,판매비용,기타영업비용,80200,기타영업비용,영업비용E,15000000,0,차변,C사
545,2024-05-29,00005,유동자산,현금및현금성자산,보통예금,10300,보통예금,영업비용E,0,15000000,대변,하나은행
546,2024-05-30,00001,유동자산,현금및현금성자산,보통예금,10300,보통예금,객실 매출,29000000,0,차변,하나은행


In [37]:
journal[journal['대분류'] == '매출']

,일자,전표번호,대분류,중분류,소분류,계정코드,계정과목,적요,차변,대변,구분,거래처명
3,2024-01-03,00001,매출,매출,객실수입,40100,객실수입,객실 매출,0,33000000,대변,호텔스닷컴
9,2024-01-04,00001,매출,매출,객실수입,40100,객실수입,객실 매출,0,25000000,대변,호텔스넷
11,2024-01-05,00001,매출,매출,객실수입,40100,객실수입,객실 매출,0,30000000,대변,호텔스닷컴
13,2024-01-06,00001,매출,매출,객실수입,40100,객실수입,객실 매출,0,25000000,대변,호텔스넷
15,2024-01-07,00001,매출,매출,객실수입,40100,객실수입,객실 매출,0,31000000,대변,호텔스닷컴
...,...,...,...,...,...,...,...,...,...,...,...,...
531,2024-05-26,00001,매출,매출,객실수입,40100,객실수입,객실 매출,0,34000000,대변,호텔스넷
533,2024-05-27,00001,매출,매출,객실수입,40100,객실수입,객실 매출,0,39000000,대변,호텔스닷컴
535,2024-05-28,00001,매출,매출,객실수입,40100,객실수입,객실 매출,0,38000000,대변,호텔스넷
537,2024-05-29,00001,매출,매출,객실수입,40100,객실수입,객실 매출,0,25000000,대변,호텔스닷컴
